<a href="https://colab.research.google.com/github/Georgeman31/TestTask_Sravni/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B8_%D0%9F%D1%80%D0%BE%D1%82%D0%B0%D1%81%D0%BE%D0%B2%D0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных

In [1]:
#Импорт библиотек
import json
import pandas as pd
import os
import datetime

In [2]:
#выгрузка данных из архива
!unzip '/content/data_example.zip'

Archive:  /content/data_example.zip
   creating: data_example/
  inflating: data_example/0.json     
  inflating: data_example/1.json     
  inflating: data_example/10.json    
  inflating: data_example/100.json   
  inflating: data_example/101.json   
  inflating: data_example/102.json   
  inflating: data_example/103.json   
  inflating: data_example/104.json   
  inflating: data_example/105.json   
  inflating: data_example/106.json   
  inflating: data_example/107.json   
  inflating: data_example/108.json   
  inflating: data_example/109.json   
  inflating: data_example/11.json    
  inflating: data_example/110.json   
  inflating: data_example/111.json   
  inflating: data_example/112.json   
  inflating: data_example/113.json   
  inflating: data_example/114.json   
  inflating: data_example/115.json   
  inflating: data_example/116.json   
  inflating: data_example/117.json   
  inflating: data_example/118.json   
  inflating: data_example/119.json   
  inflating: data_example

# Функции обработки данных

In [3]:
def read_json_data(path):
  """Считывание входных данных из json файлов

  Функция принимает на вход директорию, читает оттуда все имеющиеся json файлы,
  возвращает список словарей с данными

  """

  lst_data = []

  json_files = [filename for filename in os.listdir(path) if filename.endswith('.json')]
  for json_file in json_files:
    with open(os.path.join(path, json_file)) as path_json_file:
      lst_data.extend(json.load(path_json_file))

  return lst_data





def credit_usage_sessions_count(arrears, start_date, end_date):
  """Функция расчета количества сессий пользования кредитной карты

  На вход подается временной ряд с задолженностью, дата начала периода и дата окончания
  периода.
  Функция преобразует входной список в dataframe, обрабатывает его, выбирает период и
  считает количесто ненулевых сессий вподряд.
  Возвращает количество сессий пользования кредитной карты за указанный период

  """

  try:
    dt_start, dt_end = datetime.date.fromisoformat(start_date), datetime.date.fromisoformat(end_date)
  except:
    raise ValueError("Неверный формат даты, формат должен быть следующим: YYYY-MM-DD")

  if dt_start > dt_end:
    raise ValueError("Нижняя гранижа периода должна быть меньше верхней")


  df_data = pd.DataFrame.from_dict(arrears, orient='columns')
  df_data.dropna(how='any', inplace=True) # очищается от пропусков, тк дата без задолженности и наоборот нерепрезентативны
  df_data['calculationDate'] = pd.to_datetime(df_data['calculationDate'])
  df_data.sort_values(by='calculationDate', inplace=True)
  needed_interval = df_data.loc[df_data["calculationDate"].between(start_date, end_date)]
  series_needed_interval = needed_interval.outstanding

  return len(series_needed_interval[series_needed_interval!=0].groupby(series_needed_interval.eq(0).cumsum()).agg(len).values)

# Проверка работы на тест кейсах

In [4]:
# считывание файлов в переменную
data = read_json_data('/content/data_example')

Стандартные тест кейсы

In [5]:
print(credit_usage_sessions_count(data, '2022-10-11', '2022-11-11'))

346


In [6]:
print(credit_usage_sessions_count(data, '2017-05-01', '2017-05-30'))

95


In [7]:
print(credit_usage_sessions_count(data, '2005-12-30', '2025-01-11'))

21782


Случай выхода из диапазона доступных дат

In [8]:
print(credit_usage_sessions_count(data, '2024-12-30', '2025-01-11'))

0


Случай перепутанного порядка дат

In [9]:
print(credit_usage_sessions_count(data, '2019-07-15', '2018-09-16'))

ValueError: ignored

Случай неверного формата дат

In [10]:
print(credit_usage_sessions_count(data, '30:05:2022', '15:072022'))

ValueError: ignored